<a href="https://colab.research.google.com/github/Morax19/ML_UCV/blob/main/Proyecto1/ML_Proyecto1_DanielHernandez_JuanPRivas_JoinerRojas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PROYECTO 1: APRENDIZAJE SUPERVISADO<br>
Aprendizaje Automático, Escuela de Computación, Facultad de Ciencias - UCV
<br>
<br>
#Proyecto #1
<br>
<br>
Autores:
<br>Juan Pablo Rivas
<br>Joiner Rojas
<br>Daniel Hernández

Fecha: 28 de Febrero 2025

## Parte 1: Entendimiento y análisis de los datos

### Introducción

Breve introducción al dataset

Podríamos hacer los imports de las librerías principales acá explicando qlq con cada una

### Planteamiento del problema

Acá podemos hablar un poco del problema que vamos a buscar solucionar con nuestro modelo, me preocupa principalmente que los datasets usualmente se usan para 1 tipo de problema y necesitamos 2 para aprendizaje supervisado y no supervisado.

### Análisis exploratorio de los datos

Acá podemos hacer la inspección inicial del dataset, ver las dimensiones, valores nulos, que significa cada columna, etc

### Pre-procesamiento

En esta sección se hará las modificaciones necesarias al dataset mencionadas en el análsis inicial.

## Parte 2: Modelo de aprendizaje supervisado

Pequeña explicación de como el modelo que usaremos va a solucionar uno de los problemas planteados

### Comparativa entre modelos propuestos

#### Modelo 1

Habladera del modelo(?

1. Imports

2. Metricas a usar para medir el rendimiento

Breve explicacion de las metricas y justificacion de su uso.

3. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

4. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

#### Modelo 2

Habladera del modelo(?

1. Imports

2. Metricas a usar para medir el rendimiento

Breve explicacion de las metricas y justificacion de su uso.

3. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

4. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

#### Modelo 3

Habladera del modelo(?

1. Imports

2. Metricas a usar para medir el rendimiento

Breve explicacion de las metricas y justificacion de su uso.

3. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

4. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

### Conclusiones

Habladera de que modelo vamos a elegir al final y justificar porque, ademas de agregar la parte de almacenar el modelo

## Parte 3: Modelo de aprendizaje no supervisado

Same as above, explicación de como el modelo va a solucionar un problema planteado

### Comparativa entre modelos propuestos

#### Modelo 1

Habladera del modelo(?

1. Imports

2. Metricas a usar para medir el rendimiento

Breve explicacion de las metricas y justificacion de su uso.

3. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

4. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

#### Modelo 2

Habladera del modelo(?

1. Imports

2. Metricas a usar para medir el rendimiento

Breve explicacion de las metricas y justificacion de su uso.

3. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

4. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

#### Modelo 3

Habladera del modelo(?

1. Imports

2. Metricas a usar para medir el rendimiento

Breve explicacion de las metricas y justificacion de su uso.

3. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

4. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

### Conclusiones

Habladera de que modelo vamos a elegir al final y justificar porque, ademas de agregar la seccion correspondiente al almacenamiento del modelo

## Parte 4: Creacion de la API(?

No se si esto deba ser parte del notebook pero aca lo agrego